In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing_extensions import Annotated, List, Optional, Dict, TypedDict

from langchain_community.document_loaders import WebBaseLoader

from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.language_models import BaseChatModel
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_experimental.utilities import PythonREPL

In [3]:
import os
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
repl = PythonREPL()

os.makedirs("temp", exist_ok=True)

In [4]:
@tool
def scrape_webpages(urls: List[str]) -> str:
    '''User requests and bs4 to scrape the provided webpages for detailed information'''
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f"<Document name={doc.metadata['source']}>\n{doc.page_content}\n</Document>"
            for doc in docs
        ]
    )

@tool
def create_outline(
    points: Annotated[List[str], "List of main points or sections"],
    file_name: Annotated[str, "File path to save the outline"]
) -> Annotated[str, "Path of the saved outline file"]:
    '''Create and save an outline'''
    file_to_use = os.path.join(os.getcwd(), "temp", file_name)
    with open(file_to_use, "w") as f:
        for i, point in enumerate(points):
            f.write(f"{i+1}. {point}\n")
    return f"Outline saved to {file_to_use}"

@tool
def read_document(
    file_name: Annotated[str, "File path to read the document from"],
    start: Annotated[Optional[object], "The start line. Default is 0"] = None,
    end: Annotated[Optional[object], "The end line. Default is None"] = None
):
    '''Read the specified document. Accepts integers or strings that represent integers.'''
    def to_int_or_none(v: Optional[object]) -> Optional[int]:
        if v is None:
            return None
        # allow ints unchanged
        if isinstance(v, int):
            return v
        # allow numeric strings like "0", " 3 "
        if isinstance(v, str):
            s = v.strip()
            if s == "" or s.lower() == "none" or s.lower() == "null":
                return None
            if s.isdigit() or (s.startswith('-') and s[1:].isdigit()):
                return int(s)
            # try float-like but castable to int (not recommended but safe)
            try:
                f = float(s)
                return int(f)
            except Exception:
                raise ValueError(f"Cannot coerce start/end value to int: {v!r}")
        raise ValueError(f"Unsupported type for start/end: {type(v)}")

    start_i = to_int_or_none(start)
    end_i = to_int_or_none(end)

    file_to_use = os.path.join(os.getcwd(), "temp", file_name)
    with open(file_to_use, "r", encoding="utf-8") as f:
        lines = f.readlines()
    if start_i is None:
        start_i = 0
    if end_i is None:
        end_i = len(lines)
    # safety clamps
    start_i = max(0, start_i)
    end_i = min(len(lines), end_i)
    if start_i > end_i:
        return f"Invalid range: start ({start_i}) > end ({end_i})."

    return "\n".join(lines[start_i:end_i])

@tool
def write_document(
    content: Annotated[str, "The content to write to the document"],
    file_name: Annotated[str, "File path to save the document to"]
):
    '''Create and save a text document'''
    file_to_use = os.path.join(os.getcwd(), "temp", file_name)
    with open(file_to_use, "w") as f:
        f.write(content)
    return f"Document saved to {file_name}"

@tool
def edit_document(
    file_name: Annotated[str, "File path to read the document from and save the edited document to"],
    insert: Annotated[Dict[int, str], "A dictionary where keys are line numbers and values are the text to insert at those lines"]
):
    '''Edit a document by inserting text at specified line numbers'''
    file_to_use = os.path.join(os.getcwd(), "temp", file_name)
    with open(file_to_use, "r") as f:
        lines = f.readlines()
    
    sorted_inserts = sorted(insert.items())
    for line_num, text in sorted_inserts:
        if 1 <= line_num <= len(lines):
            lines.insert(line_num - 1, text + "\n")
        else:
            return f"Line number {line_num} is out of range for the document with {len(lines)} lines."
    with open(file_to_use, "w") as f:
        f.writelines(lines)
    return f"Document edited and saved to {file_name}"

@tool
def python_repl_tool(
        code: Annotated[str, "Python code to execute to generate your chart"]
):
    '''Use this to execute python code. If you want to see the output of any value,
    you should print it with `print(...)`. This is visible to the user'''
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Error executing code: {repr(e)}"
    return f"Successfully executed: \n ```python\n{code}\n```\n Stdout: {result}"

Define the supervisor

In [5]:
from typing_extensions import Literal


class State(MessagesState):
    next: str

def make_supervisor_node(llm: BaseChatModel, members: List[str]):
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the worker to act next, each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )
    class Router(TypedDict):
        next: str

    def supervisor(state: State) -> Command[str]:
        messages = [
            {"role": "system", "content": system_prompt}
        ] + state["messages"]

        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]

        if goto not in options:
            raise ValueError(f"Invalid route: {goto}")
        
        if goto == "FINISH":
            return Command(goto=END, update={"next": "FINISH"})
        return Command(goto=goto, update={"next": goto})
    
    return supervisor

    

Research Team

In [6]:
from langchain.chat_models import init_chat_model

In [7]:
from langchain_tavily import TavilySearch
import uuid



llm = init_chat_model("moonshotai/kimi-k2-instruct-0905",model_provider="groq")
tavily_tool = TavilySearch(max_results=3)

# Put this in a fresh cell and run it BEFORE you call create_react_agent(...)
import builtins, functools, inspect, types

# ensure uuid is available to any evaluated code
import uuid
builtins.uuid = uuid

def underlying_callable_of(tool_obj):
    """
    Return a Python callable for 'tool_obj':
    - If it's a plain function, return it.
    - If it's an object with a callable .run or .__call__, return that.
    - If it's a framework StructuredTool/BaseTool instance, try .run or .invoke or .__call__.
    - Otherwise return None.
    """
    # plain function
    if isinstance(tool_obj, types.FunctionType):
        return tool_obj
    # bound method
    if inspect.ismethod(tool_obj):
        return tool_obj
    # callable objects with __call__
    if callable(tool_obj) and hasattr(tool_obj, "__call__") and not isinstance(tool_obj, type):
        # But we still prefer an explicit .run if present (framework pattern)
        if hasattr(tool_obj, "run") and callable(getattr(tool_obj, "run")):
            return getattr(tool_obj, "run")
        return tool_obj.__call__
    # framework-wrapped tools: try .run or .invoke (common patterns)
    for attr in ("run", "invoke", "__call__"):
        if hasattr(tool_obj, attr) and callable(getattr(tool_obj, attr)):
            return getattr(tool_obj, attr)
    return None

def make_named_tool(name: str, func_or_tool):
    """
    Return a proper function object exposing __name__ and .name for LangGraph.
    If func_or_tool is an object with a .run/.invoke method, wrap that method.
    """
    fn = underlying_callable_of(func_or_tool)
    if fn is None:
        raise TypeError(f"Cannot extract callable from {func_or_tool!r}")
    @functools.wraps(fn)
    def tool_fn(*args, **kwargs):
        return fn(*args, **kwargs)
    tool_fn.__name__ = name
    tool_fn.name = name
    return tool_fn

# Build wrappers or reuse existing tool objects depending on what you have.
# For TavilySearch (third-party), use its .run / .__call__ as underlying callable:
search_tool = make_named_tool("search", tavily_tool)

# For scrape_webpages: your function is decorated with @tool; underlying_callable_of will find .run or .invoke.
# But for many @tool implementations the object *is* a StructuredTool that the framework can accept directly.
# So prefer passing the original object if it is already a framework tool type.
scrape_callable = underlying_callable_of(scrape_webpages)
if scrape_callable is None:
    # fallback: use the object itself (maybe it's a BaseTool subclass that create_react_agent accepts)
    web_scrape_tool = scrape_webpages
else:
    # create a named wrapper that exposes __name__ etc.
    web_scrape_tool = make_named_tool("web_scraper", scrape_webpages)

# Debug prints to confirm what we will pass
print("search_tool:", type(search_tool), getattr(search_tool, "__name__", None), getattr(search_tool, "name", None))
print("web_scrape_tool:", type(web_scrape_tool), getattr(web_scrape_tool, "__name__", None), getattr(web_scrape_tool, "name", None))

# Now create agents with those tool objects (the objects are either functions or framework-compatible objects)
search_agent = create_react_agent(llm, tools=[search_tool])
# If web_scrape_tool is a raw framework tool object, pass it directly
web_scraper_agent = create_react_agent(llm, tools=[web_scrape_tool])

# Recreate your nodes if needed, then include the same tool objects in invoke()
# e.g.
# response = research_graph.invoke({"messages":[message]}, tools=[search_tool, web_scrape_tool])

# sanity checks
print("search_tool:", getattr(search_tool, "__name__", None), getattr(search_tool, "name", None))
print("web_scrape_tool:", getattr(web_scrape_tool, "__name__", None), getattr(web_scrape_tool, "name", None))
def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update = {
            "messages": [HumanMessage(content = result["messages"][-1].content, name="search")
            ]
        },
        goto = "supervisor"
    )

#web_scraper_agent = create_react_agent(llm, tools = [scrape_webpages])
def web_scraper_node(state: State) -> Command[Literal["supervisor"]]:
    result = web_scraper_agent.invoke(state)
    return Command(
        update = {
            "messages": [HumanMessage(content = result["messages"][-1].content, name="web_scraper")
            ]
        },
        goto = "supervisor"
    )

research_supervisor_node = make_supervisor_node(llm, members=["search", "web_scraper"])


search_tool: <class 'function'> search search
web_scrape_tool: <class 'function'> web_scraper web_scraper
search_tool: search search
web_scrape_tool: web_scraper web_scraper


C:\Users\acer\AppData\Local\Temp\ipykernel_9272\2182689904.py:77: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  search_agent = create_react_agent(llm, tools=[search_tool])
C:\Users\acer\AppData\Local\Temp\ipykernel_9272\2182689904.py:79: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  web_scraper_agent = create_react_agent(llm, tools=[web_scrape_tool])


In [8]:
from langgraph.graph import StateGraph, START


research_builder = StateGraph(State)
research_builder.add_node("supervisor", research_supervisor_node)
research_builder.add_node("search", search_node)
research_builder.add_node("web_scraper", web_scraper_node)

research_builder.add_edge(START, "supervisor")
research_graph = research_builder.compile()



In [9]:
print("search_tool name:", getattr(search_tool, "name", None), "__name__:", getattr(search_tool, "__name__", None))
print("web_scrape_tool name:", getattr(web_scrape_tool, "name", None), "__name__:", getattr(web_scrape_tool, "__name__", None))

search_tool name: search __name__: search
web_scrape_tool name: web_scraper __name__: web_scraper


In [10]:
# Build a minimal fake State object like the supervisor would pass:
fake_state = State(messages=[{"role":"system","content":"...your system..."}, {"role":"user","content":"What is the capital of New Zealand?"}], next="search")
# Directly invoke the search_agent (this shows what it tries to call)
agent_result = search_agent.invoke(fake_state)
print(agent_result["messages"][-1].content)

I can answer this directly without needing to use the search tool. The capital of New Zealand is **Wellington**.


*Writing Team*

In [11]:
doc_writer_agent = create_react_agent(
    llm, 
    tools = [write_document, edit_document, read_document],
    prompt = (
        "You can read, write and edit documents based on note taker's outlines. "
        "Don't ask follow up questions"
    )
)

def doc_writing_node(state: State) -> Command[Literal["supervisor"]]:
    result = doc_writer_agent.invoke(state)
    return Command(
        update = {
            "messages": [
                HumanMessage(content = result["messages"][-1].content, name="doc_writer")
            ]
        },
        goto = "supervisor",
    )

note_taking_agent = create_react_agent(
    llm,
    tools = [create_outline, read_document],
    prompt = (
        "You can read documents and create outlines for the document writer."
        "Don't ask follow up questions."
    )
)

def note_taking_node(state: State) -> Command[Literal["supervisor"]]:
    result = note_taking_agent.invoke(state)
    return Command(
        update = {
            "messages": [
                HumanMessage(content = result["messages"][-1].content, name="note_taker")
            ]
        },
        goto = "supervisor",
    )

chart_generating_agent = create_react_agent(
    llm,
    tools = [read_document, python_repl_tool],

)

def chart_generating_node(state: State) -> Command[Literal["supervisor"]]:
    result = chart_generating_agent.invoke(state)
    return Command(
        update = {
            "messages": [
                HumanMessage(content = result["messages"][-1].content, name="chart_generator")
            ]
        },
        goto = "supervisor",
    )

doc_writing_supervisor = make_supervisor_node(llm, members=["note_taker", "doc_writer", "chart_generator"])

C:\Users\acer\AppData\Local\Temp\ipykernel_9272\1591720967.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  doc_writer_agent = create_react_agent(
C:\Users\acer\AppData\Local\Temp\ipykernel_9272\1591720967.py:21: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  note_taking_agent = create_react_agent(
C:\Users\acer\AppData\Local\Temp\ipykernel_9272\1591720967.py:41: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  chart_generating_agent = create_react_agent(


In [12]:
from langgraph.graph import StateGraph, START

writing_builder = StateGraph(State)
writing_builder.add_node("supervisor", doc_writing_supervisor)
writing_builder.add_node("note_taker", note_taking_node)
writing_builder.add_node("doc_writer", doc_writing_node)
writing_builder.add_node("chart_generator", chart_generating_node)

writing_builder.add_edge(START, "supervisor")
writing_graph = writing_builder.compile()

In [13]:
for s in writing_graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Write an outline for a poem about dogs and after that write the poem itself and store it"
            }
        ]
    },
    {"recursion_limit": 30}
):
    print(s)
    print("---")

{'supervisor': {'next': 'doc_writer'}}
---
{'doc_writer': {'messages': [HumanMessage(content='I\'ve created both the outline and the poem about dogs. The outline provides a structured approach to celebrating dogs through their history, characteristics, personality traits, daily life, and deeper meaning. \n\nThe poem "Ode to Dogs" is a six-part poem that follows this structure, written in rhyming verse that captures the special relationship between humans and dogs, their various qualities, and the profound impact they have on our lives. Both files have been saved for your reference.', additional_kwargs={}, response_metadata={}, name='doc_writer', id='d8f06a04-a812-4abe-880d-455a2f99ae78')]}}
---
{'supervisor': {'next': 'FINISH'}}
---


In [ ]:
teams_supervisor_node = make_supervisor_node(llm, members=["research_team", "writing_team"])

def call_research_team(state: State) -> Command[Literal["supervisor"]]:
    response = research_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update = {
            "messages": [HumanMessage(content = response["messages"][-1].content, name="research_team")]
        },
        goto = "supervisor"
    )

def call_writing_team(state: State) -> Command[Literal["supervisor"]]:
    response = writing_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update = {
            "messages": [HumanMessage(content = response["messages"][-1].content, name="writing_team")]
        },
        goto = "supervisor"
    )


super_builder = StateGraph(State)
super_builder.add_node("supervisor", teams_supervisor_node)
super_builder.add_node("research_team", call_research_team)
super_builder.add_node("writing_team", call_writing_team)

super_builder.add_edge(START, "supervisor")
super_graph = super_builder.compile()

In [16]:
from IPython.display import Markdown
Markdown(f"```mermaid\n{super_graph.get_graph().draw_mermaid()}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	supervisor(supervisor)
	research_team(research_team)
	writing_team(writing_team)
	__end__(<p>__end__</p>)
	__start__ --> supervisor;
	supervisor --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [18]:
for s in super_graph.stream(
    {
        "messages": [
            {
                "role":"user",
  "content": "Research why the gold price has been increasing recently (2024). Come up with the reasons and compile a report."
            }
        ]
    },
    {"recursion_limit": 30}
):
    print(s)
    print("---")

{'supervisor': {'next': 'research_team'}}
---
{'research_team': {'messages': [HumanMessage(content="Now I have enough information to compile a comprehensive report. Let me create a detailed analysis of the gold price increase in 2024.\n\n# Gold Price Increase Analysis Report 2024\n\n## Executive Summary\n\nGold prices experienced a significant surge in 2024, with prices rising approximately 27.23% throughout the year, reaching an all-time nominal high of $2,331 per troy ounce in April. This dramatic increase has been driven by a complex interplay of geopolitical uncertainties, central bank activities, inflation concerns, and shifting global economic dynamics.\n\n## Key Statistics\n- **Price Increase**: ~27% increase in 2024\n- **Peak Price**: $2,331 per troy ounce (April 2024)\n- **Starting Price 2024**: $2,076 per troy ounce\n- **Central Bank Purchases**: Over 1,000 tonnes for the third consecutive year\n\n## Primary Drivers of Gold Price Increases in 2024\n\n### 1. Central Bank Accum